In [64]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix
import numpy as np

In [38]:
# Load the dataset
file_path = '/content/one_hot_encoded.csv'
data = pd.read_csv(file_path).set_index('user_id')
data.head()

,'Til There Was You,1-900,101 Dalmatians,12 Angry Men,187,2 Days in the Valley,"20,000 Leagues Under the Sea",2001: A Space Odyssey,3 Ninjas: High Noon At Mega Mountain,8 1/2,...,Wolf,Wonderland,Wyatt Earp,Year of the Horse,Young Frankenstein,Young Guns,Young Guns II,Zeus and Roxanne,unkonwn,recommended_movie
user_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Starship Troopers
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Primary Colors
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,Rosencrantz and Guildenstern Are Dead
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Bean
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Dark City


In [14]:
m = n = 12
s = 19
users = data.iloc[:,:-1]
user_similarities = cosine_similarity(users)

user_filtering_candidate_set = {}

for target_user in range(len(users)):
  similar_users = np.argsort(user_similarities[target_user,:])[-m-1:-1]
  similar_users_movies = users.iloc[similar_users]
  similar_users_movies = similar_users_movies[similar_users_movies>0].dropna(axis=1, how='all')
  candidate_set = set(similar_users_movies.fillna(0).sum(axis = 0).sort_values(ascending=False)[:s].keys())
  user_filtering_candidate_set[target_user] = candidate_set

In [26]:
data

,'Til There Was You,1-900,101 Dalmatians,12 Angry Men,187,2 Days in the Valley,"20,000 Leagues Under the Sea",2001: A Space Odyssey,3 Ninjas: High Noon At Mega Mountain,8 1/2,...,Wolf,Wonderland,Wyatt Earp,Year of the Horse,Young Frankenstein,Young Guns,Young Guns II,Zeus and Roxanne,unkonwn,recommended_movie
user_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Starship Troopers
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Primary Colors
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,Rosencrantz and Guildenstern Are Dead
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Bean
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Dark City
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Hush
940,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Contact
941,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,Swingers


In [22]:
cand_ids = {}
for target_user in range(1, len(users)+1):
  y = data.loc[target_user, 'recommended_movie']
  if y in user_filtering_candidate_set[target_user-1]:

    cand_ids[target_user] = (user_filtering_candidate_set[target_user-1], y)

In [40]:
movie_ratings = data.drop(columns=['recommended_movie'])

sparse_ratings = csr_matrix(movie_ratings.values)

movie_similarity = cosine_similarity(sparse_ratings.transpose())


movie_similarity_df = pd.DataFrame(movie_similarity, index=movie_ratings.columns, columns=movie_ratings.columns)

movie_similarity_df.head(10)

,'Til There Was You,1-900,101 Dalmatians,12 Angry Men,187,2 Days in the Valley,"20,000 Leagues Under the Sea",2001: A Space Odyssey,3 Ninjas: High Noon At Mega Mountain,8 1/2,...,Witness,Wolf,Wonderland,Wyatt Earp,Year of the Horse,Young Frankenstein,Young Guns,Young Guns II,Zeus and Roxanne,unkonwn
'Til There Was You,1.000000,0.0,0.068680,0.083333,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0
1-900,0.000000,1.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0
101 Dalmatians,0.068680,0.0,1.000000,0.022893,0.06868,0.066848,0.143208,0.047904,0.0,0.000000,...,0.0,0.071734,0.000000,0.029285,0.00,0.052334,0.000000,0.058571,0.000000,0.0
12 Angry Men,0.083333,0.0,0.022893,1.000000,0.00000,0.000000,0.034752,0.135622,0.0,0.000000,...,0.0,0.000000,0.000000,0.000000,0.00,0.084667,0.030429,0.000000,0.000000,0.0
187,0.000000,0.0,0.068680,0.000000,1.00000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.000000,0.102062,0.000000,0.25,0.000000,0.000000,0.000000,0.000000,0.0
2 Days in the Valley,0.000000,0.0,0.066848,0.000000,0.00000,1.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.028239,0.000000,0.000000,0.00,0.020602,0.000000,0.000000,0.162221,0.0
"20,000 Leagues Under the Sea",0.000000,0.0,0.143208,0.034752,0.00000,0.000000,1.000000,0.145436,0.0,0.000000,...,0.0,0.072595,0.000000,0.088911,0.00,0.026481,0.076139,0.044455,0.000000,0.0
2001: A Space Odyssey,0.000000,0.0,0.047904,0.135622,0.00000,0.000000,0.145436,1.000000,0.0,0.096724,...,0.0,0.020236,0.000000,0.049568,0.00,0.177162,0.084895,0.049568,0.000000,0.0
3 Ninjas: High Noon At Mega Mountain,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,1.0,0.000000,...,0.0,0.000000,0.235702,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0
8 1/2,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.096724,0.0,1.000000,...,0.0,0.000000,0.000000,0.000000,0.00,0.035223,0.050637,0.000000,0.000000,0.0


In [44]:
def find_similar_movies(movie_title, movie_similarity_df, top_n=5):

    if movie_title in movie_similarity_df.columns:
        similar_movies = movie_similarity_df[movie_title].sort_values(ascending=False)[1:top_n+1].index.tolist()
        return similar_movies


In [61]:
def recommend_movies_item_based(candidate_set, movie_similarity_df, top_n=10):
    all_recommendations = {}
    for movie in candidate_set:
        similar_movies = find_similar_movies(movie, movie_similarity_df, top_n)

        if similar_movies is not None:
            for sim_movie in similar_movies:

                all_recommendations[sim_movie] = all_recommendations.get(sim_movie, 0) + 1


    sorted_recommendations = sorted(all_recommendations.items(), key=lambda x: x[1], reverse=True)
    return [movie for movie, _ in sorted_recommendations[:top_n]]


predicted_movies_for_users = {}
for user_id, candidate_set_info in cand_ids.items():
    candidate_set, _ = candidate_set_info
    predicted_movies = recommend_movies_item_based(candidate_set, movie_similarity_df)
    predicted_movies_for_users[user_id] = predicted_movies


In [62]:
count = 0
for user_id, (candidate, y) in cand_ids.items():
  if y in predicted_movies_for_users[user_id]:
    count += 1

In [63]:
count/len(cand_ids)

0.4925373134328358